# **MIP2-Run2**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.1/324.1 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.5 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown
import pyomo


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']



### **Generate Mathematical Model**

In [4]:
problem = """You are the production planner in a food and beverage production fascility. You are tasked with planning production for the next 12 time periods with the goal of maximizing profit. The production fascility is able to produce two types of food and three types of beverage. The food is produced in whole units, while the beverage can be produced in any fraction of a liter. For each product that is produced at period t, a fixed cost and variable cost incurred. Each product can also be stored which is also associated with a variable cost per product stored. There is also a maximum demand for each product and time period which means that a specific product will not be sold anymore if the demand is met for that time period. Please note, that the demand does not have to be met. The revenue per product and time also differs. The cost for production and storage, the demand and revenue are all provided as in sepearte csv files named "fixed_cost_production.csv", "variable_cost_production.csv", "variable_cost_storage.csv", "demand.csv" and "revenue.csv" where all files follow the structre of the columns representing time and rows representing product. Finally, there is a constant storage capacity and production capacity for each product that resets each time period. The storage capacity is 580 units of food 1, 687 units of food 2, 599L of beverage one, 788 Liters of beverage 2 and 294L of beverage 3. The production capacity is 1080 units of food 1, 908 units of food 2, 408L of beverage one, 1000L of beverage 2 and 403L of beverage 3."""

In [6]:
client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 2
)


In [7]:
print(response1.choices[0].message.content)

Parameters:
- t = Time period, t ∈ T = {1, 2, …, 12}.
- i = Product type, i ∈ I = {1, 2, 3, 4, 5}. (where 1 and 2 are food items and 3, 4, 5 are beverage items).
- FCPi = Fixed cost of production for each product type i as specified in "fixed_cost_production.csv".
- VCPi = Variable cost of production for each product type i as specified in "variable_cost_production.csv".
- VCSi = Variable cost of Storage for each product type i as specified in "variable_cost_storage.csv".
- Dit = Demand for product type i at time t as specified in "demand.csv".
- Rit = Revenue for product type i at time t as specified in "revenue.csv".
- Si_cap = Storage capacity for each product type i = {580, 687, 599, 788, 294}.
- Pi_cap = Production capacity for each product type i = {1080, 908, 408, 1000, 403}.

Decision Variables:
- Pit = Quantity of product type i to produce at time period t.
- Sit = Quantity of product type i to store at the end of time period t.

Objective Function:
Maximize total profit over 

### **Generate Pyomo Code**

In [8]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [9]:
Markdown(response2.choices[0].message.content)

Here is the code to solve the above optimization problem using Pyomo:

```python
from pyomo.environ import *

# Create a model
model = ConcreteModel()

# Load the sample data
FCPi = dict()  # fill the dictionary with data from fixed_cost_production.csv
VCPi = dict()  # fill the dictionary with data from variable_cost_production.csv
VCSi = dict()  # fill the dictionary with data from variable_cost_storage.csv
Dit = dict()  # fill the dictionary with data from demand.csv
Rit = dict()  # fill the dictionary with data from revenue.csv
Si_cap = {1: 580, 2: 687, 3: 599, 4: 788, 5: 294}  # Sample data for storage capacity
Pi_cap = {1: 1080, 2: 908, 3: 408, 4: 1000, 5: 403}  # Sample data for production capacity

# Index
model.I = Set(initialize=[i for i in range(1,6)])
model.T = Set(initialize=[t for t in range(1,13)])

# Parameters
model.Rit = Param(model.I, model.T, initialize=Rit)
model.Dit = Param(model.I, model.T, initialize=Dit)
model.FCPi = Param(model.I, initialize=FCPi)
model.VCPi = Param(model.I, initialize=VCPi)
model.VCSi = Param(model.I, initialize=VCSi)
model.Si_cap = Param(model.I, initialize=Si_cap)
model.Pi_cap = Param(model.I, initialize=Pi_cap)

# Variables
model.Pit = Var(model.I, model.T, domain=NonNegativeReals)
model.Sit = Var(model.I, model.T, domain=NonNegativeReals)

# Objective function
def obj_expression(model):
    return sum(sum(model.Rit[i, t] * model.Pit[i, t] - model.FCPi[i] * model.Pit[i, t] - model.VCPi[i] * model.Pit[i, t] - model.VCSi[i] * model.Sit[i, t] for i in model.I) for t in model.T)

model.obj = Objective(rule=obj_expression, sense=maximize)

# Constraints
def storage_balance_rule(model, i, t):
    if t == 1:
        return model.Sit[i, t] == model.Pit[i, t] - model.Dit[i, t]
    else:
        return model.Sit[i, t] == model.Pit[i, t] + model.Sit[i, t - 1] - model.Dit[i, t]

model.balance_rule = Constraint(model.I, model.T, rule=storage_balance_rule)

def production_balance_rule(model, i, t):
    if t == 1:
        return model.Pit[i, t] <= model.Dit[i, t] + model.Si_cap[i]
    else:
        return model.Pit[i, t] + model.Sit[i, t - 1] <= model.Dit[i, t] + model.Si_cap[i]

model.production_rule = Constraint(model.I, model.T, rule=production_balance_rule)

def production_capacity_rule(model, i, t):
    return model.Pit[i, t] <= model.Pi_cap[i]

model.capacity_rule = Constraint(model.I, model.T, rule=production_capacity_rule)

# Solve the model
solver = SolverFactory('glpk')
solver.solve(model)
model.pprint() 
```
Note: The sample data should be loaded from the respective csv files for real application. Here, dictionaries are used as placeholders.

### **Run the code Generated by GPT4**

In [10]:
from pyomo.environ import *

# Create a model
model = ConcreteModel()

# Load the sample data
FCPi = dict()  # fill the dictionary with data from fixed_cost_production.csv
VCPi = dict()  # fill the dictionary with data from variable_cost_production.csv
VCSi = dict()  # fill the dictionary with data from variable_cost_storage.csv
Dit = dict()  # fill the dictionary with data from demand.csv
Rit = dict()  # fill the dictionary with data from revenue.csv
Si_cap = {1: 580, 2: 687, 3: 599, 4: 788, 5: 294}  # Sample data for storage capacity
Pi_cap = {1: 1080, 2: 908, 3: 408, 4: 1000, 5: 403}  # Sample data for production capacity

# Index
model.I = Set(initialize=[i for i in range(1,6)])
model.T = Set(initialize=[t for t in range(1,13)])

# Parameters
model.Rit = Param(model.I, model.T, initialize=Rit)
model.Dit = Param(model.I, model.T, initialize=Dit)
model.FCPi = Param(model.I, initialize=FCPi)
model.VCPi = Param(model.I, initialize=VCPi)
model.VCSi = Param(model.I, initialize=VCSi)
model.Si_cap = Param(model.I, initialize=Si_cap)
model.Pi_cap = Param(model.I, initialize=Pi_cap)

# Variables
model.Pit = Var(model.I, model.T, domain=NonNegativeReals)
model.Sit = Var(model.I, model.T, domain=NonNegativeReals)

# Objective function
def obj_expression(model):
    return sum(sum(model.Rit[i, t] * model.Pit[i, t] - model.FCPi[i] * model.Pit[i, t] - model.VCPi[i] * model.Pit[i, t] - model.VCSi[i] * model.Sit[i, t] for i in model.I) for t in model.T)

model.obj = Objective(rule=obj_expression, sense=maximize)

# Constraints
def storage_balance_rule(model, i, t):
    if t == 1:
        return model.Sit[i, t] == model.Pit[i, t] - model.Dit[i, t]
    else:
        return model.Sit[i, t] == model.Pit[i, t] + model.Sit[i, t - 1] - model.Dit[i, t]

model.balance_rule = Constraint(model.I, model.T, rule=storage_balance_rule)

def production_balance_rule(model, i, t):
    if t == 1:
        return model.Pit[i, t] <= model.Dit[i, t] + model.Si_cap[i]
    else:
        return model.Pit[i, t] + model.Sit[i, t - 1] <= model.Dit[i, t] + model.Si_cap[i]

model.production_rule = Constraint(model.I, model.T, rule=production_balance_rule)

def production_capacity_rule(model, i, t):
    return model.Pit[i, t] <= model.Pi_cap[i]

model.capacity_rule = Constraint(model.I, model.T, rule=production_capacity_rule)

# Solve the model
solver = SolverFactory('glpk')
solver.solve(model)
model.pprint()

ERROR:pyomo.core:Rule failed when generating expression for Objective obj with index None:
ValueError: Error retrieving immutable Param value (Rit[(1, 1)]):
	The Param value is undefined and no default value is specified.
ERROR:pyomo.core:Constructing component 'obj' from data=None failed:
    ValueError: Error retrieving immutable Param value (Rit[(1, 1)]):
	The Param value is undefined and no default value is specified.


ValueError: Error retrieving immutable Param value (Rit[(1, 1)]):
	The Param value is undefined and no default value is specified.

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**

In [35]:
from pyomo.environ import *

# Create a model
model = ConcreteModel()

# data loaded by human
fixed_cost_production = pd.read_csv("fixed_cost_production.csv", usecols=lambda column: column != 0)
variable_cost_production = pd.read_csv("variable_cost_production.csv", usecols=lambda column: column != 0)
variable_cost_storage = pd.read_csv("variable_cost_storage.csv", usecols=lambda column: column != 0)
demand = pd.read_csv("demand.csv", usecols=lambda column: column != 0)
revenue = pd.read_csv("revenue.csv", usecols=lambda column: column != 0)

# Transform into dictionary format
FCPi = {(i, j-1): fixed_cost_production.iloc[i-1, j-1] for i in range(1, len(fixed_cost_production)+1) for j in range(2, len(fixed_cost_production.columns)+1)}
VCPi = {(i, j-1): variable_cost_production.iloc[i-1, j-1] for i in range(1, len(variable_cost_production)+1) for j in range(2, len(variable_cost_production.columns)+1)}
VCSi = {(i, j-1): variable_cost_storage.iloc[i-1, j-1] for i in range(1, len(variable_cost_storage)+1) for j in range(2, len(variable_cost_storage.columns)+1)}
Dit = {(i, j-1): demand.iloc[i-1, j-1] for i in range(1, len(demand)+1) for j in range(2, len(demand.columns)+1)}
Rit = {(i, j-1): revenue.iloc[i-1, j-1] for i in range(1, len(revenue)+1) for j in range(2, len(revenue.columns)+1)}


Si_cap = {1: 580, 2: 687, 3: 599, 4: 788, 5: 294}  # Sample data for storage capacity
Pi_cap = {1: 1080, 2: 908, 3: 408, 4: 1000, 5: 403}  # Sample data for production capacity


# Index
model.I = Set(initialize=[i for i in range(1,6)])
model.T = Set(initialize=[t for t in range(1,13)])

# Parameters
model.Rit = Param(model.I, model.T, initialize=Rit)
model.Dit = Param(model.I, model.T, initialize=Dit)
model.FCPi = Param(model.I, initialize=FCPi)
model.VCPi = Param(model.I, initialize=VCPi)
model.VCSi = Param(model.I, initialize=VCSi)
model.Si_cap = Param(model.I, initialize=Si_cap)
model.Pi_cap = Param(model.I, initialize=Pi_cap)

# Variables
model.Pit = Var(model.I, model.T, domain=NonNegativeReals)
model.Sit = Var(model.I, model.T, domain=NonNegativeReals)

# Objective function
def obj_expression(model):
    return sum(sum(model.Rit[i, t] * model.Pit[i, t] - model.FCPi[i] * model.Pit[i, t] - model.VCPi[i] * model.Pit[i, t] - model.VCSi[i] * model.Sit[i, t] for i in model.I) for t in model.T)

model.obj = Objective(rule=obj_expression, sense=maximize)

# Constraints
def storage_balance_rule(model, i, t):
    if t == 1:
        return model.Sit[i, t] == model.Pit[i, t] - model.Dit[i, t]
    else:
        return model.Sit[i, t] == model.Pit[i, t] + model.Sit[i, t - 1] - model.Dit[i, t]

model.balance_rule = Constraint(model.I, model.T, rule=storage_balance_rule)

def production_balance_rule(model, i, t):
    if t == 1:
        return model.Pit[i, t] <= model.Dit[i, t] + model.Si_cap[i]
    else:
        return model.Pit[i, t] + model.Sit[i, t - 1] <= model.Dit[i, t] + model.Si_cap[i]

model.production_rule = Constraint(model.I, model.T, rule=production_balance_rule)

def production_capacity_rule(model, i, t):
    return model.Pit[i, t] <= model.Pi_cap[i]

model.capacity_rule = Constraint(model.I, model.T, rule=production_capacity_rule)

# Solve the model
solver = SolverFactory('glpk')
solver.solve(model)
model.pprint()

ERROR:pyomo.core:Rule failed for Param 'FCPi' with index (1, 1):
KeyError: "Index '(1, 1)' is not valid for indexed component 'FCPi'"
ERROR:pyomo.core:Constructing component 'FCPi' from data=None failed:
    KeyError: "Index '(1, 1)' is not valid for indexed component 'FCPi'"


KeyError: "Index '(1, 1)' is not valid for indexed component 'FCPi'"